# Using RNN and LSTM to predict text emotion
## Abstract

This script implements a sentiment analysis pipeline using PyTorch and TorchText to process and classify the IMDB movie review dataset.     It includes preprocessing with Spacy tokenizer and GloVe embeddings, and defines RNN and LSTM neural network models for training.     The models are optimized using Adam and evaluated across multiple epochs to demonstrate effectiveness in predicting sentiments from text reviews.

**My project consists of the following three processes：**
- Data Handling and Preprocessing
- Neural Network Implementation
- Model Training and Evaluation

## Data Handling and Preprocessing
Utilizes TorchText and Spacy for loading, tokenizing, and preprocessing the IMDB movie review dataset. Sets up vocabularies with GloVe embeddings to facilitate efficient text processing.

In [3]:
import random
import spacy #用来将文本字符串分割成单独的词汇或标记
import torch #专门用来深度学习的库，提供张量的数据类型
import torch.nn as nn #用于构建神经网络
import torch.optim as optim #提供各种优化算法
from torchtext import data #NLP工具包
from torchtext.datasets import IMDB #运用了影评数据集
from tqdm import tqdm #在循环过程添进度条
#这段代码的功能是设置PyTorch的计算设备，以便在进行神经网络训练时选择合适的硬件（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) #这里查看用的哪种设备

TEXT = data.Field(tokenize='spacy', include_lengths=True, tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

# 加载IMDB数据集，创建训练集和测试集以及验证集
train_data, test_data = IMDB.splits(TEXT, LABEL) #自带分割方法，默认0.7训练集
train_data, valid_data = train_data.split(random_state=random.seed(22))

# 进一步设置了如何为文本和标签数据构建词汇表，并且涵盖了使用预训练词向量。（从训练集中提取所有唯一的单词及符号）
MAX_VOCAB_SIZE = 25000 #设置最大词汇量：重点关注数据集中频繁出现的单词
TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, vectors='glove.6B.100d') #使用了预训练词向量glove，把每个单词向量化成长度100的向量，帮助模型更好理解单词语义，提高模型处理文本能力
LABEL.build_vocab(train_data) #转化label为0.0/1.0 

# 创建训练集、验证集和测试集的迭代器，设置设备
BATCH_SIZE = 32 #一次迭代批量处理32个数据
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True, #根据文本长度排序（长或短文本之间的联系）
    device=device #用上面探测到的设备来处理
)



cuda
downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:32<00:00, 2.58MB/s]
.vector_cache/glove.6B.zip: 862MB [04:13, 3.40MB/s]                                
100%|█████████▉| 399999/400000 [00:19<00:00, 20052.09it/s]


## Neural Network Implementation
Defines and trains RNN and LSTM models using PyTorch to analyze sentiment in text data. Incorporates embedding layers, recurrent layers, and fully connected layers to process and classify textual information.

In [4]:
# 定义RNN模型和LSTM模型
class RNN(nn.Module):
    def __init__(self, inputs, embedding, hidden, outputs):
        super().__init__()
        self.embedding = nn.Embedding(inputs, embedding) #嵌入层：将文本的单词映射到高维向量的过程：根据索引input找到对应单词，embedding是每个单词向量化好之后的长度
        self.rnn = nn.RNN(embedding, hidden) 
        self.fc1 = nn.Linear(hidden, hidden // 2)
        self.fc2 = nn.Linear(hidden // 2, outputs) #设置了三层：分别是一层RNN（处理序列数据），两层全连接层（将RNN输出进行降维）
        self.relu = nn.ReLU(inplace=True) #设置了一个激活函数，提供非线性

    def forward(self, text):
        embedded = self.embedding(text) #输入文本通过嵌入层转换成嵌入向量
        output, hidden = self.rnn(embedded) #RNN处理嵌入向量，输出每个时间步输出和隐藏状态
        predictions = self.relu(self.fc1(hidden.squeeze(0))) 
        predictions = self.fc2(predictions) #隐藏状态去除张量为1的维度，通过全连接层和ReLU激活层生成最终预测
        return predictions
#如果把 RNN 网络在一个时间步的活动可视化，你会看到有 100 个输入神经元，每个神经元同时接收来自 32 个不同样本的数据。
#每个样本的数据是独立处理的，尽管使用的是相同的神经元。
#这种结构允许 RNN 能够有效地处理包含多个独立样本的批次数据，这些数据共享相同的网络权重，但各自维护自己的状态信息。

class LSTM(nn.Module):
    def __init__(self, inputs, embedding, hidden, outputs):
        super().__init__()
        self.embedding = nn.Embedding(inputs, embedding)
        self.lstm = nn.LSTM(embedding, hidden) #这里换成了LSTM模型
        self.fc1 = nn.Linear(hidden, hidden // 2)
        self.fc2 = nn.Linear(hidden // 2, outputs)
        self.relu = nn.ReLU(inplace=True)
#结构与上面RNN一样
    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        predictions = self.relu(self.fc1(hidden[-1]))  # Use the last hidden state
        predictions = self.fc2(predictions)
        return predictions



## Model Training and Evaluation
Conducts training, validation, and testing across multiple epochs, using Adam optimizer and binary cross-entropy loss. Includes functionality to predict sentiment on new reviews, demonstrating the models' effectiveness.

In [6]:
#这部分代码是一个神经网络训练流程，包括初始化模型、定义优化器和损失函数、训练和验证过程，以及准确率计算的实现。
# 初始化模型和优化器
INPUT = len(TEXT.vocab)
print(INPUT)
EMBEDDING = 100
HIDDEN = 128
OUTPUT = 1 #二分类问题，维度为1
#以上定义了文本长度、嵌入向量维度、RNN或LSTM隐藏单元数和输出层维度
model = RNN(INPUT, EMBEDDING, HIDDEN, OUTPUT)
#这里我们先初始化RNN模型进行训练
optimizer = optim.Adam(model.parameters) #Adam优化器，是自适应学习率的方法
criterion = nn.BCEWithLogitsLoss() #损失函数使用二元交叉熵损失
# 将模型和损失函数移至设备
model = model.to(device)
criterion = criterion.to(device)


# 计算准确率
def binary_accuracy(predicts, y):
    rounded_predicts = torch.round(torch.sigmoid(predicts)) #把预测值转化为0/1的标量
    correct = (rounded_predicts == y).float() #计算预测正确的个数
    accuracy = correct.sum() / len(correct) #计算准确率
    return accuracy


# 训练
def train(model, iterator, optimizer, criterion):
    model.train() #将模式设置为训练模式（为了后面的Dropout）
    epoch_loss = 0
    epoch_accuracy = 0 #初始化损失和准确率
    for batch in tqdm(iterator, desc=f'Epoch [{epoch + 1}/{EPOCHS}]', delay=0.1): #用tqdm迭代器包装训练数据迭代器，并添加进度条和描述
        optimizer.zero_grad() #对于每个批次，清除之前的梯度
        predictions = model(batch.text[0]).squeeze(1) 
        loss = criterion(predictions, batch.label)
        accuracy = binary_accuracy(predictions, batch.label)
        loss.backward() 
        optimizer.step() #自动梯度计算并更新模型参数
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item() 
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)
        #最终返回每个训练周期的平均损失和准确率

# 验证函数，与训练相比不包含反向传播和参数更新
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_accuracy = 0
    with torch.no_grad(): #这里标明不进行梯度计算
        for batch in iterator:
            predictions = model(batch.text[0]).squeeze(1)
            loss = criterion(predictions, batch.label)
            accuracy = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)



25002


In [9]:
#接下来是一个完整的训练、验证、测试和使用模型预测的循环过程。
EPOCHS = 10 #这里设定进行十轮训练与验证并输出结果
for epoch in range(EPOCHS):
    train_loss, train_accuracy = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Accuracy: {train_accuracy * 100:.2f}%')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation Accuracy: {valid_accuracy * 100:.2f}%')

# 测试模型：对模型泛化能力最终检验
test_loss, test_accuracy = evaluate(model, test_iterator, criterion)
print(f'\nTest Loss: {test_loss:.3f} | Test Acc: {test_accuracy * 100:.2f}%')


# 创建预测模型
def predict_sentiment(model, sentence, threshold=0.5):
    model.eval() #将模型设置为评估模式（Dropout 不会丢弃任何单元，BatchNorm 会使用整个训练集的统计数据而不是单个批次的）
    nlp = spacy.load('en_core_web_sm')
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] #对要预测的文本进行分词
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] #将词汇转换为整数索引
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) #将索引转换为张量并添加一个维度
    prediction = torch.sigmoid(model(tensor)) #预测出结果并处理输出
    print(f'Proba: {prediction.item()}')
    return 'positive' if prediction.item() > threshold else 'negative'
    #打印出模型预测的概率及预测结果

# 这里运用了两个实例进行预测
positive_comment = "What a fantastic film!"
negative_comment = "I was disappointed by this movie."
print(f'Comment: {positive_comment}\nPrediction: {predict_sentiment(model, positive_comment)}')
print(f'Comment: {negative_comment}\nPrediction: {predict_sentiment(model, negative_comment)}')


Epoch [1/10]:   1%|          | 6/547 [00:00<00:18, 29.35it/s]Epoch [1/10]: 100%|██████████| 547/547 [00:17<00:00, 31.36it/s]


	Train Loss: 0.617 | Train Accuracy: 62.43%
	Validation Loss: 0.674 | Validation Accuracy: 59.86%


Epoch [2/10]: 100%|██████████| 547/547 [00:19<00:00, 28.34it/s]


	Train Loss: 0.599 | Train Accuracy: 64.61%
	Validation Loss: 0.681 | Validation Accuracy: 60.58%


Epoch [3/10]: 100%|██████████| 547/547 [00:20<00:00, 26.65it/s]


...

	Train Loss: 0.539 | Train Accuracy: 70.83%
	Validation Loss: 0.657 | Validation Accuracy: 63.36%


Epoch [9/10]: 100%|██████████| 547/547 [00:16<00:00, 33.72it/s]


	Train Loss: 0.516 | Train Accuracy: 73.83%
	Validation Loss: 0.618 | Validation Accuracy: 70.46%


Epoch [10/10]: 100%|██████████| 547/547 [00:16<00:00, 33.11it/s]


	Train Loss: 0.477 | Train Accuracy: 77.58%
	Validation Loss: 0.647 | Validation Accuracy: 68.25%

Test Loss: 0.715 | Test Acc: 67.27%
Proba: 0.6270339488983154
Comment: What a fantastic film!
Prediction: positive
Proba: 0.8357884287834167
Comment: I was disappointed by this movie.
Prediction: positive


In [10]:
#由于以上RNN模型预测结果不理想，对模型进行了调整，是模型更复杂
class RNN(nn.Module):
    def __init__(self, inputs, embedding, hidden, outputs, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(inputs, embedding)
        self.rnn = nn.RNN(embedding, hidden, num_layers=2, bidirectional=True, dropout=dropout) #添加了双向和多层RNN
        self.fc1 = nn.Linear(hidden * 2, hidden // 2)
        self.fc2 = nn.Linear(hidden // 2, outputs)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) #加入了Dropout，防止模型过拟合

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        hidden = hidden.view(hidden.size(1), -1)  # Flatten the hidden state
        predictions = self.relu(self.fc1(self.dropout(hidden)))
        predictions = self.fc2(predictions)
        return predictions

optimizer = optim.Adam(model.parameters(), lr=0.0001) #将优化器的学习率设置成0.0001
criterion = nn.BCEWithLogitsLoss()
# 将模型和损失函数移至设备
model = model.to(device)
criterion = criterion.to(device)

# 训练模型
EPOCHS = 20 #改为了20个训练周期（epochs）
for epoch in range(EPOCHS):
    train_loss, train_accuracy = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Accuracy: {train_accuracy * 100:.2f}%')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation Accuracy: {valid_accuracy * 100:.2f}%')

# 测试模型
test_loss, test_accuracy = evaluate(model, test_iterator, criterion)
print(f'\nTest Loss: {test_loss:.3f} | Test Acc: {test_accuracy * 100:.2f}%')


# 使用模型进行预测
def predict_sentiment(model, sentence, threshold=0.5):
    model.eval()
    nlp = spacy.load('en_core_web_sm')
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    print(f'Proba: {prediction.item()}')
    return 'positive' if prediction.item() > threshold else 'negative'


# 示例预测
positive_comment = "What a fantastic film!"
negative_comment = "I was disappointed by this movie."
print(f'Comment: {positive_comment}\nPrediction: {predict_sentiment(model, positive_comment)}')
print(f'Comment: {negative_comment}\nPrediction: {predict_sentiment(model, negative_comment)}')

Epoch [1/20]: 100%|██████████| 547/547 [00:16<00:00, 33.18it/s]


	Train Loss: 0.456 | Train Accuracy: 78.30%
	Validation Loss: 0.630 | Validation Accuracy: 70.39%


Epoch [2/20]: 100%|██████████| 547/547 [00:18<00:00, 29.34it/s]


	Train Loss: 0.431 | Train Accuracy: 80.26%
	Validation Loss: 0.628 | Validation Accuracy: 71.14%


Epoch [3/20]: 100%|██████████| 547/547 [00:18<00:00, 29.29it/s]


...

	Train Loss: 0.334 | Train Accuracy: 85.81%
	Validation Loss: 0.618 | Validation Accuracy: 73.65%


Epoch [19/20]: 100%|██████████| 547/547 [00:17<00:00, 31.98it/s]


	Train Loss: 0.326 | Train Accuracy: 86.43%
	Validation Loss: 0.611 | Validation Accuracy: 73.70%


Epoch [20/20]: 100%|██████████| 547/547 [00:18<00:00, 28.81it/s]


	Train Loss: 0.323 | Train Accuracy: 86.77%
	Validation Loss: 0.614 | Validation Accuracy: 73.95%

Test Loss: 0.650 | Test Acc: 72.81%
Proba: 0.4937834143638611
Comment: What a fantastic film!
Prediction: negative
Proba: 0.40905529260635376
Comment: I was disappointed by this movie.
Prediction: negative


In [13]:
#这里我们使用LSTM模型进行训练
model = LSTM(INPUT, EMBEDDING, HIDDEN, OUTPUT)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
# 将模型和损失函数移至设备
model = model.to(device)
criterion = criterion.to(device)

EPOCHS = 20 #进行20个训练周期
for epoch in range(EPOCHS):
    train_loss, train_accuracy = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Accuracy: {train_accuracy * 100:.2f}%')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation Accuracy: {valid_accuracy * 100:.2f}%')

# 测试模型
test_loss, test_accuracy = evaluate(model, test_iterator, criterion)
print(f'\nTest Loss: {test_loss:.3f} | Test Acc: {test_accuracy * 100:.2f}%')


# 使用模型进行预测
def predict_sentiment(model, sentence, threshold=0.5):
    model.eval()
    nlp = spacy.load('en_core_web_sm')
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    print(f'Proba: {prediction.item()}')
    return 'positive' if prediction.item() > threshold else 'negative'


# 示例预测
positive_comment = "What a fantastic film!"
negative_comment = "I was disappointed by this movie."
print(f'Comment: {positive_comment}\nPrediction: {predict_sentiment(model, positive_comment)}')
print(f'Comment: {negative_comment}\nPrediction: {predict_sentiment(model, negative_comment)}')


Epoch [1/20]:   0%|          | 2/547 [00:00<00:42, 12.75it/s]Epoch [1/20]: 100%|██████████| 547/547 [00:20<00:00, 26.22it/s]


	Train Loss: 0.680 | Train Accuracy: 55.98%
	Validation Loss: 0.668 | Validation Accuracy: 59.06%


Epoch [2/20]: 100%|██████████| 547/547 [00:21<00:00, 25.71it/s]


	Train Loss: 0.682 | Train Accuracy: 54.64%
	Validation Loss: 0.683 | Validation Accuracy: 60.09%


Epoch [3/20]: 100%|██████████| 547/547 [00:21<00:00, 25.31it/s]


...

	Train Loss: 0.008 | Train Accuracy: 99.80%
	Validation Loss: 0.649 | Validation Accuracy: 88.05%


Epoch [19/20]: 100%|██████████| 547/547 [00:17<00:00, 31.31it/s]


	Train Loss: 0.007 | Train Accuracy: 99.85%
	Validation Loss: 0.676 | Validation Accuracy: 88.26%


Epoch [20/20]: 100%|██████████| 547/547 [00:19<00:00, 28.29it/s]


	Train Loss: 0.002 | Train Accuracy: 99.98%
	Validation Loss: 0.691 | Validation Accuracy: 88.37%

Test Loss: 0.837 | Test Acc: 86.17%
Proba: 0.9982360601425171
Comment: What a fantastic film!
Prediction: positive
Proba: 0.00921712163835764
Comment: I was disappointed by this movie.
Prediction: negative
